In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.
C:\Users\amitv\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amitv\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amitv\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\amitv\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(227, 227, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [3]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(227, 227),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='binary')

Found 30000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [5]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.01)

In [6]:
model.fit_generator(
        train_generator,
#         steps_per_epoch=30000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
#         validation_steps=10000 // batch_size,
        callbacks=[es])
model.save_weights('first_try.h5')


Epoch 1/50
1875/1875 [==============================] - 264s 141ms/step - loss: 0.2903 - accuracy: 0.9396 - val_loss: 0.1864 - val_accuracy: 0.9846
Epoch 2/50
1875/1875 [==============================] - 268s 143ms/step - loss: 0.2145 - accuracy: 0.9622 - val_loss: 0.0998 - val_accuracy: 0.9825
Epoch 3/50
1875/1875 [==============================] - 268s 143ms/step - loss: 0.1622 - accuracy: 0.9735 - val_loss: 0.0585 - val_accuracy: 0.9843
